1. VRP 전체를 몇개의 차량으로   
2. CVRP 차량의 용량 제한   
3. VRPTW 고객 방문 가능 시간대   

OPEN tour   
시작 지점으로 돌아올 필요가 없는 경로(생산 경로)    
가상 경로 도입해 잘라버리면 최적 생산 순서    

데이터 생성 

In [29]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    #data 딕셔너리 생성 
    data = {}

    #data['distance_matrix][i][j]: i에서 j까지 거리
    data["distance_matrix"] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]

    data["num_vehicles"] = 1
    data["depot"] = 0
    return data
data=create_data_model()

routing 

In [30]:
#색인 관리자, 일종의 정보 관리자
manager= pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),#전체 노드 수(고객 수+출발지)
    data['num_vehicles'], #사용할 차량 수
    data['depot'] #출발지 노드 인덱스
)
#솔버 내부 인덱스를 사용자 위치 번호로 변환하는 등 인덱스를 관리함

#Routing : 경로 최적화를 위한 핵심 솔버 객체 
routing= pywrapcp.RoutingModel(manager)

#두 점 사이의 거리를 반환한다
def distance_callback(from_index,to_index):
    #내부 routing 인덱스를 실제 노드 번호로 변환 
    from_node=manager.IndexToNode(from_index)
    to_node=manager.IndexToNode(to_index)
    #거리 행령에서 두 노드 간 거리 반환 
    return data['distance_matrix'][from_node][to_node]

#거리 계산 함수를 callback 함수로 등록하고 활용함
transit_callback_index=routing.RegisterTransitCallback(distance_callback) #함수를 솔버 내부에서 사용할 수 있는 형태로 등록 
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index) #솔버에게 경로(Arc)의 비용이 거리 기반으로 계산되게 설정함 

해 찾기   
[다른 전략](https://developers.google.com/optimization/routing/routing_options?hl=ko#first_solution_strategy)

In [31]:
#시작해를 찾기 위한 휴리스틱 메소드를 등록
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy=(
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    #시작점으로부터 가까운 노드를 추가하는 방식 
)
#최적해 출력 위한 함수 
def print_solution(manager,routing,solution):
    #총 거리 
    print(f'Objective : {solution.ObjectiveValue()}miles')
    #시작 노드 인덱스
    index= routing.Start(0)
    plan_output='Route for vehicle 0:\n'
    #총 경로 거리 누적값을 저장할 변수 
    route_distance=0
    #현재 인덱스가 vehicle의 경로 종로점인지 확인하는 반복문 
    while not routing.IsEnd(index):
        #현재 노드 번호를 경로 출력 문자열에 추가 
        plan_output+=f'{manager.IndexToNode(index)} ->'
        previous_index=index
        #다음 노드 인덱스를 솔루션에서 가져온다 
        index=solution.Value(routing.NextVar(index))
        #이전 노드부터 현재 노드까지 거리를 누적한다 
        route_distance += routing.GetArcCostForVehicle(previous_index,index,0)
    #마지막 노드를 경로 출력에 추가 
    plan_output+=f'{manager.IndexToNode(index)}\n'
    #전체 경로 정보를 출력 문자열에 추가 
    plan_output+=f'Route distance: {route_distance}miles\n'
    print(plan_output)
#문제풀이 실행 
solution= routing.SolveWithParameters(search_parameters) #설정한 파라미터로 최적 경로를 계산 
if solution: # 솔루션이 존재할 경우 결과 출력 
    print_solution(manager,routing,solution)#인덱스, 솔버, 솔루션 

Objective : 7293miles
Route for vehicle 0:
0 ->7 ->2 ->3 ->4 ->12 ->6 ->8 ->1 ->11 ->10 ->5 ->9 ->0
Route distance: 7293miles



해 경로를 리스트에 저장하기 

In [33]:
def get_routes(solution,routing,manager):
    routes=[]
    for route_nbr in range(routing.vehicles()):
        index=routing.Start(route_nbr)
        route=[manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index=solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes
routes=get_routes(solution,routing,manager)
for i, route in enumerate(routes):
    print('Route',i, route)

Route 0 [0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0]
